In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [17]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoading().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [18]:
ABILoading().get_abi_by_filename(abi_path)

{'_format': 'hh-sol-artifact-1',
 'contractName': 'UniswapV2Pair',
 'sourceName': 'contracts/uniswapv2/UniswapV2Pair.sol',
 'abi': [{'inputs': [],
   'stateMutability': 'nonpayable',
   'type': 'constructor'},
  {'anonymous': False,
   'inputs': [{'indexed': True,
     'internalType': 'address',
     'name': 'owner',
     'type': 'address'},
    {'indexed': True,
     'internalType': 'address',
     'name': 'spender',
     'type': 'address'},
    {'indexed': False,
     'internalType': 'uint256',
     'name': 'value',
     'type': 'uint256'}],
   'name': 'Approval',
   'type': 'event'},
  {'anonymous': False,
   'inputs': [{'indexed': True,
     'internalType': 'address',
     'name': 'sender',
     'type': 'address'},
    {'indexed': False,
     'internalType': 'uint256',
     'name': 'amount0',
     'type': 'uint256'},
    {'indexed': False,
     'internalType': 'uint256',
     'name': 'amount1',
     'type': 'uint256'},
    {'indexed': True,
     'internalType': 'address',
     'nam

In [4]:
reorg_mon = JSONRPCReorganisationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()

In [5]:
start = start - 10

In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:60,634,171 tx:0x2bea83ef8c3766a55a16f5db2a3b2bc5513ee3770cac6a19b0974c57c6424100
Swap at block:60,634,171 tx:0xb8fc651e508af9c711ca843b681d77dd022bfe25c05e00e3924970fbb94599f6
Swap at block:60,634,172 tx:0xf1972885dfaf2bc9bab2be4202a34b1b4ad10b927caf3589cec801a0a1eaa95f
Swap at block:60,634,173 tx:0x9915b6f0901b0c93ac966dafc01f734bb888cd63206c0a7f5abb5538eb9dfd28
Swap at block:60,634,174 tx:0xbc8970a91250f63a45ee00a2a182e65acd336040cb68d0a4094ef6349bd5b2b4
Swap at block:60,634,175 tx:0x9b4e3b008a31bad3bd811284c99615ea801026b45285a215d43eca59a77fc9fd
Swap at block:60,634,175 tx:0xb5fefa8499db0b70c1777704437b73db22e2504825498605d0c231234b6b937d
Swap at block:60,634,176 tx:0x8af3917d397757aaeed6bad48695a3687f7b3ead8f6c37b5eac91858e03ce9d0
Swap at block:60,634,176 tx:0x8e643499879b1ffc8f8c209b1e26e97cf678c12248d8f38257b6b554e6fc74aa
Swap at block:60,634,176 tx:0x8e643499879b1ffc8f8c209b1e26e97cf678c12248d8f38257b6b554e6fc74aa
Swap at block:60,634,176 tx:0x8e643499879b1ffc8f8c

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0x2bea83ef8c3766a55a16f5db2a3b2bc5513ee3770cac...,60634171,1723756849,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0x032d86f50ee9c0c361f268bd7c08013465e29968,0xb8fc651e508af9c711ca843b681d77dd022bfe25c05e...,60634171,1723756849,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0xe62ec2e799305e0d367b0cc3ee2cda135bf89816,0xf1972885dfaf2bc9bab2be4202a34b1b4ad10b927caf...,60634172,1723756851,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x39eaa90a70e8fdc04e1f63db04e1c62c9ace0641,0x9915b6f0901b0c93ac966dafc01f734bb888cd63206c...,60634173,1723756853,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0xa8fb745919d959fbbdfab71b88e82bb4e6ba502e,0xbc8970a91250f63a45ee00a2a182e65acd336040cb68...,60634174,1723756855,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0x9b4e3b008a31bad3bd811284c99615ea801026b45285...,60634175,1723756857,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0xdc9232e2df177d7a12fdff6ecbab114e2231198d,0xb5fefa8499db0b70c1777704437b73db22e250482549...,60634175,1723756857,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0xf6b87181bf250af082272e3f448ec3238746ce3d,0x8af3917d397757aaeed6bad48695a3687f7b3ead8f6c...,60634176,1723756861,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0xe82635a105c520fd58e597181cbf754961d51e3e,0x8e643499879b1ffc8f8c209b1e26e97cf678c12248d8...,60634176,1723756861,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0xf0e020a4675b4776a3bb22843494ce6b42ebfa0a,0x8e643499879b1ffc8f8c209b1e26e97cf678c12248d8...,60634176,1723756861,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x882df4b0fb50a229c3b4124eb18c759911485bfb',
  'tx_hash': '0x2bea83ef8c3766a55a16f5db2a3b2bc5513ee3770cac6a19b0974c57c6424100',
  'blk_num': 60634171,
  'timestamp': 1723756849,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0x07Ff4e06865de4934409Aa6eCea503b08Cc1C78d',
   'amount0In': 2672591223194272000,
   'amount1Out': 87698214}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x032d86f50ee9c0c361f268bd7c08013465e29968',
  'tx_hash': '0xb8fc651e508af9c711ca843b681d77dd022bfe25c05e00e3924970fbb94599f6',
  'blk_num': 60634171,
  'timestamp': 1723756849,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xa5E0829CaCEd8fFDD4De3c43696c57F7D7A678ff',
   'token1': '0xB1e8B5aA98391bD7000FDE66BFEF5a9883BA914d',
   'amount